# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [ ]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.7 MB/s eta 0:00:00


We'll also get the "star of the show" today, which is Ragas!

In [ ]:
!pip install -qU ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [ ]:
!pip install -qU faiss_cpu pymupdf pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 19.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.23.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [ ]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 54 (delta 15), reused 20 (delta 7), pack-reused 8
Receiving objects: 100% (54/54), 51.28 MiB | 20.12 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [ ]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

In [ ]:
documents[0].dict()

{'page_content': '1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \nCOMPLAINT \n \nIRELL & MANELLA LLP \nMorgan Chu (SBN 70446) \nAlan Heinrich (SBN 212782) \nIian Jablon (SBN 205458) \nAbigail Sellers (SBN 342380) \nJustin Koo (SBN 351547) \nHenry White (SBN 351549) \n1800 Avenue of the Stars, Suite 900 \nLos Angeles, CA 90067 \nTelephone: (310) 277-1010 \nFax: (310) 203-7199 \nmchu@irell.com \naheinrich@irell.com  \nijablon@irell.com \nasellers@irell.com \njkoo@irell.com \nhwhite@irell.com \n \n \nAttorneys for Plaintiff Elon Musk \nSUPERIOR COURT OF CALIFORNIA \nIN AND FOR THE COUNTY OF SAN FRANCISCO \nELON MUSK, an individual, \nPlaintiff, \nvs. \nSAMUEL ALTMAN, an individual, GREGORY \nBROCKMAN, an individual, OPENAI, INC., a \ncorporation, OPENAI, L.P., a limited \npartnership, OPENAI, L.L.C., a limited liability \ncompany, OPENAI GP, L.L.C., a limited \nliability company, OPENAI OPCO, LLC, a \

In [ ]:
documents[12].page_content

'1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 13 – \nCOMPLAINT \n \na neutral group looking to collaborate widely and shift the dialog towards being about humanity \nwinning rather than any particular group or company. (I think that’s the best way to bootstrap \nourselves into being a leading research institution.).” Optimistic about the possibility of a neutral AI \nresearch group focused on humanity rather than profit for any particular individual or group, \nMr. Musk told Mr. Brockman that he would commit funding. \n54. \nMr. Musk came up with the name of the new lab, a name reflecting the Founding \nAgreement: “Open AI Institute,” or simply, “OpenAI.”  \n55. \nWith these principles of the Founding Agreement in mind, Mr. Musk joined forces \nwith Mr. Altman and Mr. Brockman to formalize and launch the project. Mr. Musk was actively \ninvolved in the project, even prior to it being publicly 

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [ ]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [ ]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!


### **ANSWER:
FAISS makes searching stuff faster by techniques like squishing data into simpler forms (vector quantization), smart ways to figure out how close or far things are (the optimized distance algorithms), and using some neat graph methods called HNSW and NSG to really speed things up.

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [ ]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [ ]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [ ]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [ ]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [ ]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [ ]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.


### **ANSWER:
- Start with a user question that comes in.
- Use that question both as the "question" itself and to fetch some related info (context) by searching through a database or some content (that's the base_retriever part).
- After fetching, just pass along this fetched context as it is for the next step, without changing or using it for any calculation right then.
- Now, take the original question and the context we just passed through, and use both to ask a more detailed question to a large language model.
- Finally, keep hold of the detailed context from before while grabbing the AI's answer to our detailed question, ready for whatever comes next.






Let's test it out!

In [ ]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [ ]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, accounting, and demand for a jury trial.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [ ]:
type(documents)

list

In [ ]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

In [ ]:
eval_documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

In [ ]:
type(eval_documents)

list

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?


### **ANSWER:

- To avoid bias torwards using chinks having same size
- If we use simialr sizes the we will bias the result from the geenratation in various of more simple chunks
- Need to use chunks of different sizes between synthetic data vs actual data

In [ ]:
len(eval_documents)

159

In [ ]:
len(documents)

159

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(eval_documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

embedding nodes:   0%|          | 0/318 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).


### **ANSWER:
The mapping refers to the distrubution of various question types. For instance, in the case presented, it's defined that half/50% of the questions are to be straightforward questions, a quarter/25% are intended for reasoning, and the remaining quarter/25% are categorized as "multi-context" questions.


It is effetively a breakdown of the type of questions

Let's look at the output and see what we can learn about it!

In [ ]:
testset.test_data[0]

DataRow(question='How would the proposed business model redefine venture capitalism in California and beyond?', contexts=['business model were valid, it would radically redefine how venture capitalism is practiced in \nCalifornia and beyond. Rather than start out as a for-profit entity from the outset, “smart” investors \nwould establish non-profits, use pre-tax donations to fund research and development, and then once \ntheir technology had been developed and proven, would slide the resulting IP assets into a new for-\nprofit venture to enrich themselves and their profit-maximizing corporate partners. That is not \nsupposed to be how the law works in California or anywhere else in this country, and this should \nnot be the first Court to hold otherwise. \n114.'], ground_truth='If the proposed business model were valid, it would radically redefine how venture capitalism is practiced in California and beyond.', evolution_type='simple')

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [ ]:
test_df = testset.to_pandas()

In [ ]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,How would the proposed business model redefine...,"[business model were valid, it would radically...","If the proposed business model were valid, it ...",simple,True
1,What strategy video game did OpenAI compete in?,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,True
2,What are OpenAI's two key principles for AGI p...,[profit developing AGI for the benefit of huma...,OpenAI's two key principles for AGI profit dev...,reasoning,True
3,How did OpenAI demonstrate their expertise in ...,[77. \nInitial work at OpenAI followed much in...,OpenAI demonstrated their expertise in a strat...,reasoning,True
4,"""What technique showcased the reasoning abilit...",[implementation for others to build on. \n84. ...,chain-of-thought prompting,multi_context,True
5,"""What strategy video game did OpenAI excel in,...",[77. \nInitial work at OpenAI followed much in...,OpenAI excelled in the strategy video game Dot...,multi_context,True
6,"""What role did the ""175 billion parameters"" pl...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,multi_context,True
7,"""What algorithm and model combination achieved...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The algorithm called the Transformer achieved ...,multi_context,True
8,What concerns does Mr. Musk have about AGI's i...,[18. \nMr. Musk has long recognized that AGI p...,Mr. Musk has concerns about AGI posing a grave...,multi_context,True
9,How did OpenAI use a deep neural network in th...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,simple,True


In [ ]:
print(test_df.iloc[8].evolution_type)
print(test_df.iloc[8].question)
print(test_df.iloc[8].ground_truth)
print(test_df.iloc[8].contexts)

multi_context
What concerns does Mr. Musk have about AGI's impact on humanity and potential misuse?
Mr. Musk has concerns about AGI posing a grave threat to humanity and falling into the wrong hands.
['18. \nMr. Musk has long recognized that AGI poses a grave threat to humanity—perhaps \nthe greatest existential threat we face today. His concerns mirrored those raised before him by \nluminaries like Stephen Hawking and Sun Microsystems founder Bill Joy. Our entire economy is \nbased around the fact that humans work together and come up with the best solutions to a hard task. \nIf a machine can solve nearly any task better than we can, that machine becomes more economically \nuseful than we are. As Mr. Joy warned, with strong AGI, “the future doesn’t need us.” Mr. Musk', 'to its mission to develop AGI for the benefit of humanity, not to personally benefit the individual \nDefendants and the largest technology company in the world.  \nDetailed Allegations \nA. \nMr. Musk’s Concerns Over 

In [ ]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()
test_evolution_type = test_df["evolution_type"].values.tolist()

In [ ]:
test_evolution_type

['simple',
 'simple',
 'reasoning',
 'reasoning',
 'multi_context',
 'multi_context',
 'multi_context',
 'multi_context',
 'multi_context',
 'simple']

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [ ]:
test_questions

['How would the proposed business model redefine venture capitalism in California and beyond?',
 'What strategy video game did OpenAI compete in?',
 "What are OpenAI's two key principles for AGI profit development?",
 'How did OpenAI demonstrate their expertise in a strategy video game more complex than chess?',
 '"What technique showcased the reasoning ability of large language models, as demonstrated by researchers at Google and further explored by researchers at the University of Tokyo and Google, using OpenAI\'s GPT-3 and a modification called chain-of-thought prompting?"',
 '"What strategy video game did OpenAI excel in, like DeepMind did in chess, shogi, and Go?"',
 '"What role did the "175 billion parameters" play in OpenAI\'s GPT-3 model and its impact on future models?"',
 '"What algorithm and model combination achieved superhuman performance in a challenging task and addressed deep learning limitations in understanding long text sequences?"',
 "What concerns does Mr. Musk hav

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [ ]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths,
    "evolution_type" : test_evolution_type
})

In [ ]:
response_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth', 'evolution_type'],
    num_rows: 10
})

In [ ]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,How would the proposed business model redefine...,"[business model were valid, it would radically...","If the proposed business model were valid, it ...",simple,True
1,What strategy video game did OpenAI compete in?,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,True
2,What are OpenAI's two key principles for AGI p...,[profit developing AGI for the benefit of huma...,OpenAI's two key principles for AGI profit dev...,reasoning,True
3,How did OpenAI demonstrate their expertise in ...,[77. \nInitial work at OpenAI followed much in...,OpenAI demonstrated their expertise in a strat...,reasoning,True
4,"""What technique showcased the reasoning abilit...",[implementation for others to build on. \n84. ...,chain-of-thought prompting,multi_context,True
5,"""What strategy video game did OpenAI excel in,...",[77. \nInitial work at OpenAI followed much in...,OpenAI excelled in the strategy video game Dot...,multi_context,True
6,"""What role did the ""175 billion parameters"" pl...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,multi_context,True
7,"""What algorithm and model combination achieved...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The algorithm called the Transformer achieved ...,multi_context,True
8,What concerns does Mr. Musk have about AGI's i...,[18. \nMr. Musk has long recognized that AGI p...,Mr. Musk has concerns about AGI posing a grave...,multi_context,True
9,How did OpenAI use a deep neural network in th...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,simple,True


In [ ]:
id = 9
print(response_dataset[id]['question'])
response_dataset[id]['evolution_type']

How did OpenAI use a deep neural network in their research on text generation?


'simple'

Let's take a peek and see what that looks like!

In [ ]:
response_dataset[0]

{'question': 'How would the proposed business model redefine venture capitalism in California and beyond?',
 'answer': 'If the proposed business model were valid, it would redefine venture capitalism by having investors establish non-profits, use pre-tax donations for research and development, and then transfer the resulting IP assets into a for-profit venture to enrich themselves and their corporate partners.',
 'contexts': ['business model were valid, it would radically redefine how venture capitalism is practiced in \nCalifornia and beyond. Rather than start out as a for-profit entity from the outset, “smart” investors \nwould establish non-profits, use pre-tax donations to fund research and development, and then once \ntheir technology had been developed and proven, would slide the resulting IP assets into a new for-\nprofit venture to enrich themselves and their profit-maximizing corporate partners. That is not \nsupposed to be how the law works in California or anywhere else in t

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [ ]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
results.items()

dict_items([('faithfulness', 0.9), ('answer_relevancy', 0.9241544797713932), ('context_recall', 1.0), ('context_precision', 0.9083333332711112), ('answer_correctness', 0.8529140659596595)])

In [ ]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,evolution_type,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How would the proposed business model redefine...,"If the proposed business model were valid, it ...","[business model were valid, it would radically...","If the proposed business model were valid, it ...",simple,1.0,0.912065,1.0,1.000000,0.726505
1,What strategy video game did OpenAI compete in?,Dota 2,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,1.0,0.954052,1.0,1.000000,0.716374
2,What are OpenAI's two key principles for AGI p...,OpenAI's two key principles for AGI profit dev...,[profit developing AGI for the benefit of huma...,OpenAI's two key principles for AGI profit dev...,reasoning,1.0,0.999998,1.0,0.833333,0.998785
3,How did OpenAI demonstrate their expertise in ...,OpenAI demonstrated their expertise in a strat...,[77. \nInitial work at OpenAI followed much in...,OpenAI demonstrated their expertise in a strat...,reasoning,1.0,0.950209,1.0,1.000000,1.000000
4,"""What technique showcased the reasoning abilit...",Chain-of-thought prompting,[implementation for others to build on. \n84. ...,chain-of-thought prompting,multi_context,1.0,0.869085,1.0,0.916667,0.996956
5,"""What strategy video game did OpenAI excel in,...",Dota 2,[a superhuman level of play in the games of ch...,OpenAI excelled in the strategy video game Dot...,multi_context,0.0,0.885973,1.0,0.500000,0.968325
6,"""What role did the ""175 billion parameters"" pl...","The ""175 billion parameters"" played a signific...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,multi_context,1.0,0.965980,1.0,0.833333,0.911763
7,"""What algorithm and model combination achieved...",The Transformer algorithm was created at Googl...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The algorithm called the Transformer achieved ...,multi_context,1.0,0.811109,1.0,1.000000,0.736256
8,What concerns does Mr. Musk have about AGI's i...,Mr. Musk has concerns that AGI poses a grave t...,[18. \nMr. Musk has long recognized that AGI p...,Mr. Musk has concerns about AGI posing a grave...,multi_context,1.0,0.951636,1.0,1.000000,0.735466
9,How did OpenAI use a deep neural network in th...,OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,simple,1.0,0.941439,1.0,1.000000,0.738710


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [ ]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [ ]:
print(response["answer"])

The plaintiff is Elon Musk.


In [ ]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [ ]:
print(response["answer"])

The complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices, and seeking accounting, restitution, disgorgement of funds, and an injunction against the Defendants. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [ ]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths,
    "evolution_type" : test_evolution_type

})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [ ]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
advanced_retrieval_results

{'faithfulness': 0.8417, 'answer_relevancy': 0.9478, 'context_recall': 1.0000, 'context_precision': 0.9533, 'answer_correctness': 0.8363}

In [ ]:
(advanced_retrieval_results_df['answer_correctness']).mean()


0.836287765741208

In [ ]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,evolution_type,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How would the proposed business model redefine...,The proposed business model would redefine ven...,"[business model were valid, it would radically...","If the proposed business model were valid, it ...",simple,1.000000,0.909852,1.0,1.000000,0.865008
1,What strategy video game did OpenAI compete in?,"OpenAI competed in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,1.000000,1.000000,1.0,1.000000,0.741034
2,What are OpenAI's two key principles for AGI p...,OpenAI's two key principles for AGI profit dev...,[profit developing AGI for the benefit of huma...,OpenAI's two key principles for AGI profit dev...,reasoning,0.000000,1.000000,1.0,0.750000,0.998786
3,How did OpenAI demonstrate their expertise in ...,OpenAI demonstrated their expertise in a strat...,[77. \nInitial work at OpenAI followed much in...,OpenAI demonstrated their expertise in a strat...,reasoning,0.750000,0.971075,1.0,1.000000,0.842019
4,"""What technique showcased the reasoning abilit...",The technique that showcased the reasoning abi...,[implementation for others to build on. \n84. ...,chain-of-thought prompting,multi_context,1.000000,0.913904,1.0,0.950000,0.964533
5,"""What strategy video game did OpenAI excel in,...","OpenAI excelled in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,OpenAI excelled in the strategy video game Dot...,multi_context,1.000000,0.920092,1.0,1.000000,0.740220
6,"""What role did the ""175 billion parameters"" pl...","The ""175 billion parameters"" in OpenAI's GPT-3...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,multi_context,1.000000,0.965067,1.0,0.833333,0.895534
7,"""What algorithm and model combination achieved...","The algorithm called the Transformer, which is...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The algorithm called the Transformer achieved ...,multi_context,0.666667,0.933997,1.0,1.000000,0.741583
8,What concerns does Mr. Musk have about AGI's i...,Mr. Musk has concerns about AGI falling into t...,[18. \nMr. Musk has long recognized that AGI p...,Mr. Musk has concerns about AGI posing a grave...,multi_context,1.000000,0.959677,1.0,1.000000,0.739610
9,How did OpenAI use a deep neural network in th...,OpenAI used a deep neural network by pre-train...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,simple,1.000000,0.904463,1.0,1.000000,0.834551


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [ ]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9242, 'context_recall': 1.0000, 'context_precision': 0.9083, 'answer_correctness': 0.8529}

And see how our advanced retrieval modified our chain!

In [ ]:
advanced_retrieval_results

{'faithfulness': 0.8417, 'answer_relevancy': 0.9478, 'context_recall': 1.0000, 'context_precision': 0.9533, 'answer_correctness': 0.8363}

In [ ]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.900000,0.841667,-0.058333
1,answer_relevancy,0.924154,0.947813,0.023658
2,context_recall,1.000000,1.000000,0.000000
3,context_precision,0.908333,0.953333,0.045000
4,answer_correctness,0.852914,0.836288,-0.016626


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [ ]:
# use `text-embedding-3-small` embedding model from OpenAI
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# Initialize a vector store/index using open source FAISS with the document and
# embedding model
vector_store = FAISS.from_documents(documents, new_embeddings)

In [ ]:
# Create a retriever for the vector store
new_retriever = vector_store.as_retriever()

In [ ]:
# Initialize a Multi query retriever with the previously created retriever and our LLM
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [ ]:
# Set up a retrieval chain combining our document chain (integrating our document into the context)
# with our advanced retriever also the document stuffing value from document chain
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [ ]:
# Interate through test questions, using our retrieval chain on each to compile answers and contexts

answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [ ]:
# create a dataset using the `Dataset` from HF

new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths,
    "evolution_type" : test_evolution_type

})

In [ ]:
# evaluate
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Reciew the retrieval result dict
new_advanced_retrieval_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9422, 'context_recall': 1.0000, 'context_precision': 0.9417, 'answer_correctness': 0.8025}

In [ ]:
# Prepare our dataframe on one table to compare which performs better

df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.900000,0.841667,1.000000,0.158333,0.100000
1,answer_relevancy,0.924154,0.947813,0.942213,-0.005599,0.018059
2,context_recall,1.000000,1.000000,1.000000,0.000000,0.000000
3,context_precision,0.908333,0.953333,0.941667,-0.011667,0.033333
4,answer_correctness,0.852914,0.836288,0.802470,-0.033817,-0.050444


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

### **ANSWER:
Based on the metrics above, it's apparent that Text Embedding 3 achieves the highest faithfulness score at 1.00, marking improvement over ADA (0.842) and Baseline (0.9), with an increase of 0.158 and 0.1 respectively. However, it shows a marginal decline in answer relevancy and context precision compared to ADA, and a notable decrease in answer correctness from both Baseline and ADA, with deltas of -0.0338 and -0.0504 respectively. This suggests that while Text Embedding 3 significantly enhances the faithfulness of the generated text, it may slightly compromise on relevancy, precision, and correctness in comparison to its predecessors.

Considering these deltas, TE3 shows mixed results. Its overall advantage depends on the priority given to different metrics.

Yes, `text-embedding-3-small` is better than `ada` but I cannot see a significant based solely on the data above.


## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [ ]:
### YOUR CODE HERE

In [ ]:
# retrieve the pandas dataframe
new_advanced_retrieval_results_df = new_advanced_retrieval_results.to_pandas()
new_advanced_retrieval_results_df.head()

,question,answer,contexts,ground_truth,evolution_type,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How would the proposed business model redefine...,The proposed business model would involve esta...,"[business model were valid, it would radically...","If the proposed business model were valid, it ...",simple,1.0,0.897573,1.0,1.000000,0.813875
1,What strategy video game did OpenAI compete in?,"OpenAI competed in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,1.0,0.953843,1.0,1.000000,0.741034
2,What are OpenAI's two key principles for AGI p...,OpenAI's two key principles for AGI profit dev...,[profit developing AGI for the benefit of huma...,OpenAI's two key principles for AGI profit dev...,reasoning,1.0,1.000000,1.0,1.000000,0.998786
3,How did OpenAI demonstrate their expertise in ...,OpenAI demonstrated their expertise in a strat...,[77. \nInitial work at OpenAI followed much in...,OpenAI demonstrated their expertise in a strat...,reasoning,1.0,0.980716,1.0,0.750000,0.838564
4,"""What technique showcased the reasoning abilit...",The technique that showcased the reasoning abi...,[implementation for others to build on. \n84. ...,chain-of-thought prompting,multi_context,1.0,0.913904,1.0,0.833333,0.712365


In [ ]:


# Retrieve only the multi-context
results_df_multi_context = results_df[results_df["evolution_type"] == "multi_context"]
advanced_retrieval_results_df_multi_context = advanced_retrieval_results_df[advanced_retrieval_results_df["evolution_type"] == "multi_context"]
new_advanced_retrieval_results_df_multi_context = new_advanced_retrieval_results_df[new_advanced_retrieval_results_df["evolution_type"] == "multi_context"]



In [ ]:

# convert back into dict, take the average scores
results_df_multi_context_dict = results_df_multi_context[
    [ 'faithfulness','answer_relevancy'	,'context_recall',	'context_precision',	'answer_correctness']
    ].mean().to_dict()

advanced_retrieval_results_df_multi_context_dict = advanced_retrieval_results_df_multi_context[
    ['faithfulness','answer_relevancy'	,'context_recall',	'context_precision',	'answer_correctness']
    ].mean().to_dict()

new_advanced_retrieval_results_df_multi_context_dict = new_advanced_retrieval_results_df_multi_context[
    ['faithfulness','answer_relevancy'	,'context_recall',	'context_precision',	'answer_correctness']
    ].mean().to_dict()



In [ ]:
# sample output
results_df_multi_context_dict

{'faithfulness': 0.8,
 'answer_relevancy': 0.8967564984814942,
 'context_recall': 1.0,
 'context_precision': 0.8499999999638888,
 'answer_correctness': 0.869753327222584}

In [ ]:
# sample df
pd.DataFrame(list(results_df_multi_context_dict.items()), columns=['Metric', 'Baseline'])

,Metric,Baseline
0,faithfulness,0.800000
1,answer_relevancy,0.896756
2,context_recall,1.000000
3,context_precision,0.850000
4,answer_correctness,0.869753


In [ ]:
# Prepare our dataframe on one table to compare which performs better

df_baseline = pd.DataFrame(list(results_df_multi_context_dict.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results_df_multi_context_dict.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results_df_multi_context_dict.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged



,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.800000,0.933333,1.000000,0.066667,0.200000
1,answer_relevancy,0.896756,0.938547,0.935720,-0.002827,0.038963
2,context_recall,1.000000,1.000000,1.000000,0.000000,0.000000
3,context_precision,0.850000,0.956667,0.966667,0.010000,0.116667
4,answer_correctness,0.869753,0.816296,0.758522,-0.057774,-0.111231


#### Text Embedding 3 outperforms ADA and Baseline in faithfulness (1.00 vs. 0.933 vs. 0.80), context precision (0.967 vs. 0.957 vs. 0.85), but underperforms in answer correctness (0.759 vs. 0.816 vs. 0.87)
